# `log_*` methods

mintoでは`log_*`というメソッドでデータを記録することができます。

このセクションでは基本的なメソッドといくつかの便利なメソッドを紹介します。

## 基本的な`log`メソッド

`minto`では2つのspaceがあり、それぞれのspaceに`DataStore`というクラスの形でデータが保存されています。詳しくは (MINTO Spaces)[minto_datastore.ipynb] セクションを参照してください。

Experiment spaceに保存する場合は

```
expriment.log_global_*
```
メソッドを使い、Run spaceに保存する場合は

```
run.log_*
```

メソッドを用います。`*` となっているところには `DataStore`のattributeに対応した名前を入れることができます。

基本的な`log`メソッドの一覧は以下です。

|data type|Experiment space | Run space|
|--|--|--|
|scalars (int \| float \| str) | log_global_parameter | log_parameter|
|`jm.Problem` | log_global_problem | log_problem|
|`ommx.v1.Instance` | log_global_instance | log_instance|
|`ommx.v1.Solution` | log_global_solution | log_solution|
|`ommx.v1.SampleSet` | log_global_sampleset | log_sampleset|
|JSON Serializable object (dict)| log_global_object | log_object |

## params

複数のスカラーを保存したい場合は`log_params`メソッドを用いてdictで複数を同時に保存することができます。

In [17]:
import minto

experiment = minto.Experiment("test", auto_saving=False, verbose_logging=False)

param_a = [1, 2, 3, 4]
param_b = [2, 3, 4, 5]

for a, b in zip(param_a, param_b):
    with experiment.run() as run:
        run.log_params({
            "a": a,
            "b": b
        })

In [5]:
experiment.get_run_table()

metadata        parameter   
       elapsed_time run_id         a  b
run_id                                 
0          0.000231      0         1  2
1          0.000062      1         2  3
2          0.000037      2         3  4
3          0.000019      3         4  5

`Experiment`spaceでも同様に複数のパラメータを同時に保存できます。

In [8]:
experiment.log_global_params({"a": a, "b": b})

experiment.get_experiment_tables()["parameter"]

,python_version,os_name,platform_info,a,b
parameter,"3.11.11 (main, Jan 14 2025, 23:36:41) [Clang 1...",Darwin,macOS-15.6.1-arm64-arm-64bit,4,5


# `solver`

最適化を行う関数をラップして入力と出力をそのまま保存することが可能です。

`log_solver`は渡した関数と同じ振る舞いをする`Callable`なオブジェクトを返します。そのオブジェクトを呼び出すと、元の関数が実行され、その入力と出力がRun spaceに保存されます。`log_solver`の`exclude_params`にrunに保存したくない引数名のリストを渡すことも可能です。例えばrunの中でinstanceのような大きいデータは保存したくない場合などに便利です。

[Quick start](quickstart.ipynb) セクションで紹介した例を`log_solver`を用いて書き直すと以下のようになります。


In [15]:
import ommx_pyscipopt_adapter as scip_ad
from ommx.dataset import miplib2017

instance = miplib2017("reblock115")

def scip_solver(instance, time_limit):
    adapter = scip_ad.OMMXPySCIPOptAdapter(instance)
    scip_model = adapter.solver_input

    scip_model.setParam("limits/time", time_limit)
    scip_model.optimize()

    return adapter.decode(scip_model)

time_limit_list = [0.5, 1.0]

experiment = minto.Experiment("test", auto_saving=False, verbose_logging=False)

for time_limit in time_limit_list:
    with experiment.run() as run:
        _solver = run.log_solver(scip_solver, exclude_params=["instance"])
        solution = _solver(instance, time_limit=time_limit)


In [16]:
experiment.get_run_table()

metadata           parameter             \
       elapsed_time run_id  solver_name time_limit   
run_id                                               
0          0.636714      0  scip_solver        0.5   
1          1.058023      1  scip_solver        1.0   

       solution_scip_solver_result                                    \
                          feasible                name     objective   
run_id                                                                 
0                             True  scip_solver_result  0.000000e+00   
1                             True  scip_solver_result -2.929282e+07   

                                    
       optimality relaxation start  
run_id                              
0               0          0  None  
1               0          0  None

## まとめ

`log_*`メソッドを用いることで、Experiment spaceとRun spaceに様々なデータを簡単に保存することができます。
`params`と`solver`は便利なパッラーとして使えるのでぜひ活用してみてください。